## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

!pip3 install -U scikit-learn<br>
!pip3 install pandas<br>
!pip3 install numpy<br>
!pip3 install matplotlib<br>
!pip3 install statsmodels<br>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 데이터 다운로드 

In [2]:
import pandas as pd
df = pd.read_csv('heart.csv')
print(df.head())
print(df.shape)

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0  
(918, 12)


## Input, Feature 설정

In [3]:
Label = df['HeartDisease']
InputFeature = df[["Age", "RestingBP", "Cholesterol", #"FastingBS", 
                   "MaxHR"]]

In [4]:
InputFeature.head()

,Age,RestingBP,Cholesterol,MaxHR
0,40,140,289,172
1,49,160,180,156
2,37,130,283,98
3,48,138,214,108
4,54,150,195,122


In [5]:
InputFeature.shape

(918, 4)

## Keras Logit 모델 fitting

In [6]:
model = Sequential()
model.add(Dense(2, activation='linear', input_shape=(4,)))
model.add(Dense(4, activation='linear'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

Metal device set to: Apple M1


2022-07-08 20:55:40.291345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-08 20:55:40.291514: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Keras 모델 살펴보기

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 4)                 12        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


## Call-back 함수
## 모델 학습

In [8]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='Models/{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [9]:
model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

Epoch 1/100


2022-07-08 20:55:40.461988: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/306 [..............................] - ETA: 2:09 - loss: 68.4958 - accuracy: 0.3333

2022-07-08 20:55:40.703622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


303/306 [============================>.] - ETA: 0s - loss: 3.0572 - accuracy: 0.5171
Epoch 1: loss improved from inf to 3.03404, saving model to Models/001-3.0340-0.5185.hdf5
306/306 [==============================] - 2s 6ms/step - loss: 3.0340 - accuracy: 0.5185 - lr: 0.0100
Epoch 2/100
298/306 [============================>.] - ETA: 0s - loss: 0.8604 - accuracy: 0.5671
Epoch 2: loss improved from 3.03404 to 0.85672, saving model to Models/002-0.8567-0.5686.hdf5
306/306 [==============================] - 2s 6ms/step - loss: 0.8567 - accuracy: 0.5686 - lr: 0.0100
Epoch 3/100
304/306 [============================>.] - ETA: 0s - loss: 0.6619 - accuracy: 0.6557
Epoch 3: loss improved from 0.85672 to 0.66418, saving model to Models/003-0.6642-0.6547.hdf5
306/306 [==============================] - 2s 5ms/step - loss: 0.6642 - accuracy: 0.6547 - lr: 0.0100
Epoch 4/100
298/306 [============================>.] - ETA: 0s - loss: 0.6206 - accuracy: 0.6544
Epoch 4: loss improved from 0.66418 to 0

Epoch 31/100
301/306 [============================>.] - ETA: 0s - loss: 0.5952 - accuracy: 0.6811
Epoch 31: loss did not improve from 0.59084
306/306 [==============================] - 2s 5ms/step - loss: 0.5950 - accuracy: 0.6797 - lr: 0.0033
Epoch 32/100
303/306 [============================>.] - ETA: 0s - loss: 0.5964 - accuracy: 0.6865
Epoch 32: loss did not improve from 0.59084
306/306 [==============================] - 2s 5ms/step - loss: 0.5963 - accuracy: 0.6863 - lr: 0.0033
Epoch 33/100
297/306 [============================>.] - ETA: 0s - loss: 0.5971 - accuracy: 0.6846
Epoch 33: loss did not improve from 0.59084

Epoch 33: ReduceLROnPlateau reducing learning rate to 0.0026214396581053737.
306/306 [==============================] - 2s 5ms/step - loss: 0.5974 - accuracy: 0.6841 - lr: 0.0033
Epoch 34/100
300/306 [============================>.] - ETA: 0s - loss: 0.5885 - accuracy: 0.6756
Epoch 34: loss improved from 0.59084 to 0.58916, saving model to Models/034-0.5892-0.6765.hd

Epoch 61/100
303/306 [============================>.] - ETA: 0s - loss: 0.5845 - accuracy: 0.6975
Epoch 61: loss did not improve from 0.58042

Epoch 61: ReduceLROnPlateau reducing learning rate to 0.0005497557576745749.
306/306 [==============================] - 2s 5ms/step - loss: 0.5831 - accuracy: 0.6993 - lr: 6.8719e-04
Epoch 62/100
302/306 [============================>.] - ETA: 0s - loss: 0.5810 - accuracy: 0.6898
Epoch 62: loss did not improve from 0.58042
306/306 [==============================] - 2s 5ms/step - loss: 0.5812 - accuracy: 0.6906 - lr: 5.4976e-04
Epoch 63/100
297/306 [============================>.] - ETA: 0s - loss: 0.5805 - accuracy: 0.6914
Epoch 63: loss did not improve from 0.58042
306/306 [==============================] - 2s 5ms/step - loss: 0.5813 - accuracy: 0.6917 - lr: 5.4976e-04
Epoch 64/100
305/306 [============================>.] - ETA: 0s - loss: 0.5826 - accuracy: 0.6863
Epoch 64: loss did not improve from 0.58042

Epoch 64: ReduceLROnPlateau reducin

301/306 [============================>.] - ETA: 0s - loss: 0.5788 - accuracy: 0.6944
Epoch 90: loss improved from 0.57832 to 0.57829, saving model to Models/090-0.5783-0.6950.hdf5
306/306 [==============================] - 2s 5ms/step - loss: 0.5783 - accuracy: 0.6950 - lr: 1.4412e-04
Epoch 91/100
301/306 [============================>.] - ETA: 0s - loss: 0.5803 - accuracy: 0.6977
Epoch 91: loss improved from 0.57829 to 0.57806, saving model to Models/091-0.5781-0.6993.hdf5
306/306 [==============================] - 2s 5ms/step - loss: 0.5781 - accuracy: 0.6993 - lr: 1.4412e-04
Epoch 92/100
301/306 [============================>.] - ETA: 0s - loss: 0.5784 - accuracy: 0.6988
Epoch 92: loss did not improve from 0.57806
306/306 [==============================] - 2s 5ms/step - loss: 0.5786 - accuracy: 0.6983 - lr: 1.4412e-04
Epoch 93/100
301/306 [============================>.] - ETA: 0s - loss: 0.5796 - accuracy: 0.7010
Epoch 93: loss did not improve from 0.57806
306/306 [================

# Model Load 하기 전, hdf5 파일 이름 꼭 확인하기

In [10]:
model.load_weights("Models/100-0.5780-0.6961.hdf5")

## FLASK 셋팅하기

In [11]:
from flask import Flask
from flask import render_template
from flask import request

In [12]:
app = Flask(__name__)

# FLASK API 구현부분

In [13]:
@app.route('/')
@app.route('/HeartDisease')
def HeartDiseasePrediction():
    Age = request.args.get("Age")
    RestingBP = request.args.get("RestingBP")
    Cholesterol = request.args.get("Cholesterol")
    MaxHR = request.args.get("MaxHR")
    
       
    if Age == None or RestingBP == None:
        return render_template('HeartDisease.html', Output = '')
    
    Input = pd.DataFrame({
        'Age': [ float(Age) ],
        'RestingBP': [ float(RestingBP) ],
        'Cholesterol': [ float(Cholesterol) ],
        'MaxHR': [ float(MaxHR) ]
    }) #진석이 바부
    ModelOutput = model.predict(Input)[0][0]

    return render_template('HeartDisease.html', Output = ModelOutput * 100)

# Flask, port 5000으로 실행

In [14]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


OSError: [Errno 48] Address already in use

In [ ]:
# http://127.0.0.1:5000/HeartDisease?Age=40&RestingBP=180&Cholesterol=200&MaxHR=180